In [5]:
import string

import xlwings as xw
import pandas as pd
import string

from src.excel import ExcelWorker
from src.parsing import Parser
from src.validation import SearchAmbiguity

In [4]:
def column_from_title(sheet: xw.Sheet) -> dict[str: str]:
    titles: list[str] = sheet.range('A1').expand('right').value
    letters: list[str] = list(string.ascii_uppercase)
    return {title: letter for title, letter in zip(titles, letters)}

In [2]:
sheet = xw.sheets['Ведомость']
col = ExcelWorker.column_from_title(sheet)
for cell in ExcelWorker.selected_cells():
    name = sheet.range(f"{col['Наименование']}{cell.row}").value
    is_shrub = Parser.get_specie(name).is_shrub
    sheet.range(f"{col['Кустарник']}{cell.row}").value = int(is_shrub)

In [9]:
sheet = xw.sheets['Ведомость']
col = ExcelWorker.column_from_title(sheet)
for cell in ExcelWorker.selected_cells():
    number = sheet.range(f"{col['Номер точки']}{cell.row}").value
    quantity = sheet.range(f"{col['Количество']}{cell.row}").value
    height = sheet.range(f"{col['Высота']}{cell.row}").value
    diameter = sheet.range(f"{col['Толщина']}{cell.row}").value
    is_shrub = bool(sheet.range(f"{col['Кустарник']}{cell.row}").value)
    problem = SearchAmbiguity.check_in_row_from_taxation_list(number, quantity, height, diameter, is_shrub)
    sheet.range(f"{col['Валидация']}{cell.row}").value = int(not problem)

In [10]:
for cell in ExcelWorker.selected_cells():
    if ',' in str(cell.value):
        cell.value = str(cell.value).replace(',', '.')